In [6]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

# # 모델 초기화
# model = ChatOpenAI(
#     temperature=0,  # 응답 다양성 설정, 0~2, 기본값은 0.7
#     model='gpt-3.5-turbo'
# )

from langchain_community.llms import Ollama

# Ollama 서버가 localhost의 11434 포트에서 실행 중이라고 가정
model = Ollama(
    model="llama3",
    temperature=0
)

# 프롬프트 템플릿 정의
restaurant_template = """
You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
{input}
"""

transport_template = """
You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
{input}
"""

destination_template = """
You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
{input}
"""

# 프롬프트 정보 리스트
prompt_infos = [
    {
        "name": "restaurants",
        "description": "Good for recommending restaurants around the tourist destinations",
        "prompt_template": restaurant_template
    },
    {
        "name": "transport",
        "description": "Good for guiding the transport to get the place",
        "prompt_template": transport_template
    },
    {
        "name": "destination",
        "description": "Good for recommending place to tour",
        "prompt_template": destination_template
    }
]

# 각 체인을 생성하고 딕셔너리에 추가
destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt = PromptTemplate.from_template(prompt_info["prompt_template"])
    chain = LLMChain(llm=model, prompt=prompt, verbose=True)
    destination_chains[name] = chain

# 기본 체인 설정
default_prompt = PromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=model, prompt=default_prompt)

# 라우터 프롬프트 템플릿 설정
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# Router Chain 초기화
router_chain = LLMRouterChain.from_llm(model, router_prompt)

# MultiPromptChain 설정
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )


- 레스토랑 추천 관련 질문

In [7]:
response = chain.run("What is the best restaurant in Seoul?")
print(response)

# response = chain.run("Can you recommend a good sushi place in Tokyo?")
# print(response)

# response = chain.run("Where can I find the best pasta in Rome?")
# print(response)



> Entering new MultiPromptChain chain...
restaurants: {'input': 'What are some good restaurants near Gyeongbokgung Palace in Seoul?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
What are some good restaurants near Gyeongbokgung Palace in Seoul?


> Finished chain.

> Finished chain.
Gyeongbokgung Palace is one of the most iconic landmarks in Seoul, isn't it? There are plenty of great restaurants around that area to satisfy your cravings. Here are some recommendations:

1. **Jongno Myeonok**: This cozy restaurant serves traditional Korean dishes like naengmyeon (cold buckwheat noodles) and jeyuk bokkeum (stir-fried pork). It's a 5-minute walk from the palace.

Address: 34, Jong-ro 3-ga, Jongno-gu, Seoul

2. **Gwangjang Market**: This historic market is just across the street from Gyeongbokgung Palace and offers a vari

- 교통 안내 관련 질문

In [8]:
response = chain.run("How can I get to the Eiffel Tower from Charles de Gaulle airport?")
print(response)

# response = chain.run("What is the best way to travel from JFK airport to Times Square?")
# print(response)

# response = chain.run("Is there a direct train from London to Edinburgh?")
# print(response)



> Entering new MultiPromptChain chain...
transport: {'input': 'What are my transportation options to get to Eiffel Tower from Charles de Gaulle airport?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
What are my transportation options to get to Eiffel Tower from Charles de Gaulle airport?


> Finished chain.

> Finished chain.
Bonjour! I'd be happy to help you with that.

From Charles de Gaulle Airport (CDG) to the Eiffel Tower, you have several transportation options. Here are a few:

1. **RER B Train**: The most convenient and cost-effective way is to take the RER B train from CDG to Paris city center. From the airport, follow the signs to the RER B station (Terminal 2). Take the train towards Saint-Rémy-lès-Chevreuse and get off at Denfert-Rochereau or Cité Universi

- 관광지 추천 관련 질문

In [9]:
response = chain.run("What are the must-see places in Paris?")
print(response)

# response = chain.run("Can you suggest some good tourist spots in New York City?")
# print(response)

# response = chain.run("What are the best attractions to visit in Tokyo?")
# print(response)



> Entering new MultiPromptChain chain...
destination: {'input': 'What are some popular tourist attractions in Paris?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
What are some popular tourist attractions in Paris?


> Finished chain.

> Finished chain.
Bonjour! As a tourist guide, I'm delighted to share with you some of the most popular and iconic tourist attractions in the City of Light - Paris!

1. **The Eiffel Tower**: The iron lady of Paris, this iconic tower is a must-visit attraction. You can take the stairs or elevator to the top for breathtaking views of the city.
2. **The Louvre Museum**: Home to the Mona Lisa and many other famous artworks, the Louvre is one of the world's largest and most visited museums.
3. **Notre-Dame Cathedral**: This beautiful Gothic cathedral has been a symbol of Paris since the 12th century. Take a m

- 기타 질문 (기본 체인으로 처리)
    - 기본 체인은 특정 카테고리에 속하지 않는 질문들을 처리합니다.

In [10]:
response = chain.run("What is the capital of France?")
print(response)

# response = chain.run("Tell me about the history of the Great Wall of China.")
# print(response)

# response = chain.run("Who won the Nobel Prize in Literature in 2020?")
# print(response)



> Entering new MultiPromptChain chain...
transport: {'input': 'How do I get to Paris, the capital of France?'}

> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
How do I get to Paris, the capital of France?


> Finished chain.

> Finished chain.
Bonjour! Getting to Paris, the City of Light, is relatively easy and convenient. As your tourist guide, I'd be happy to help you navigate the options.

There are several ways to reach Paris, depending on your starting location and preferred mode of transportation. Here are a few:

1. **By Air**: The easiest way to get to Paris is by flying into Charles de Gaulle Airport (CDG) or Orly Airport (ORY). Both airports are well-connected to the city center via public transportation:
	* RER B train from CDG to Gare du Nord station in centra